<h2>File paths and imports<h2>

In [8]:
import sys 
sys.path.append("C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code")

from chimplib.imports import *
from chimplib.recognition_utils import *

#Path to the root of the ChimpRec database ".../Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
chimprec_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
chimprec_train_dataset = f"{chimprec_dataset_path}/train"
chimprec_test_dataset = f"{chimprec_dataset_path}/test"
#Path to the root of the ChimpRec model ".../Chimprec Dataset/Models/recognition/ChimpRec"
chimprec_models_fc_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Models/recognition/ChimpRec/facenet_14_layers_fc.pth"

#Path to the root of the CCR database ".../Chimprec Dataset/Datasets/Face Recognition/CCR"
ccr_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/CCR"
ccr_train_dataset = f"{ccr_dataset_path}/train"
ccr_test_dataset = f"{ccr_dataset_path}/test"
#Path to the root of the CCR model ".../Chimprec Dataset/Models/recognition/CCR"
#ccr_models_fc_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Models/recognition/CCR/facenet_13_layers_triplet_loss.pth"
ccr_models_fc_path = "C:/Users/julie/Documents/Unif/Test_mémoire/FCL_models_CCR3/facenet_15_layers_fc.pth" #"C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code/Recognition/FaceNet/Fully_connected_layer/facenet_14_layers_fc.pth"

<h2>Utils<h2>

In [2]:
#Transformation to be applied to network input images
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# @inputs:
# model_path: path to the saved model file (.pth file) containing weights
# dataset_train_path: path to the training dataset
# device: torch device ('cpu' or 'cuda') to perform computation
# has_faces_label: boolean indicating whether face labels (.txt files) are used for cropping
# @outputs:
# model: the trained model loaded and ready for inference
# class_names: list of class names corresponding to identity labels
def load_model(model_path, dataset_train_path, device, has_label_face):
    num_classes = len(os.listdir(dataset_train_path))
    model = CustomFCLFaceNet(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    train_dataset = FullyConnectedLayerDataset(dataset_train_path, transform=transform, has_faces_label=has_label_face)
    class_names = list(train_dataset.class_to_idx.keys())

    return model, class_names


In [4]:
# @inputs:
# class_names: list of class names corresponding to identity labels
# test_set_path: path to the test dataset
# model: model used for prediction
# label_fn: function that extracts the ground-truth identity from the image filename
# has_face_label: boolean indicating whether face label files (.txt) are used for cropping (not used in this function)
# @output:
# accuracy: float value representing the proportion of correctly classified images in the test dataset
def evaluate_model(class_names, test_set_path, model, label_fn, has_face_label):
    correctly_classified = 0
    imgs = os.listdir(test_set_path)
    for img in imgs:
        img_path = os.path.join(test_set_path, img)
        image = Image.open(img_path).convert("RGB")

        predicted_label, _ = predict_face_with_fc(image, model, transform, device)
        predicted_identity = class_names[predicted_label]

        real_identity = label_fn(img)
        if predicted_identity == real_identity:
            correctly_classified += 1

    accuracy = correctly_classified / len(imgs)
    return accuracy

<h2>Evaluation on the test set<h2>

<h3>ChimpRec<h3>

In [8]:
#To switch from the file name (for example: “AD1.jpg”) to its identity (for example: “AD”)
chimprec_label_fn = lambda img: img[:2] 

chimprec_model, chimprec_class_names = load_model(chimprec_models_fc_path, chimprec_train_dataset, device, True)
chimprec_accuracy = evaluate_model(chimprec_class_names, chimprec_test_dataset, chimprec_model, chimprec_label_fn)
print(f"ChimpRec Accuracy: {chimprec_accuracy:.4f}")

132
AD1.png
AD2.png
AD3.png
AD4.png
AD5.png
AD6.png
AD7.png
BL1.png
BL3.png
BL5.png
BL6.png
BL7.png
BL8.png
BL9.png
BS1.png
BS10.png
BS2.png
BS4.png
BS5.png
BS7.png
BS8.png
BS9.png
DK1.png
DK4.png
DK5.png
DK6.png
DK7.png
DK8.png
DK9.png
IV2.png
IV3.png
IV4.png
IV6.png
IV7.png
IV8.png
IV9.png
JJ1.png
JJ2.png
JJ4.png
JJ5.png
JJ6.png
JJ7.png
JJ8.png
JJ9.png
KG1.png
KG2.png
KG3.png
KG4.png
KG5.png
KM1.png
KM2.png
KM3.png
KM4.png
KM5.png
KM6.png
KM7.png
LM1.png
LM2.png
LM3.png
LM4.png
LM5.png
LM6.png
LM7.png
LM8.png
LM9.png
MG1.png
MG11.png
MG13.png
MG2.png
MG3.png
MG4.png
MG5.png
MG7.png
MG8.png
MK1.png
MK2.png
MK3.png
MK4.png
MK5.png
MZ1.png
MZ2.png
MZ3.png
MZ4.png
MZ5.png
MZ6.png
MZ7.png
MZ8.png
NJ10.png
NJ11.png
NJ12.png
NJ13.png
NJ3.png
NJ4.png
NJ6.png
NJ7.png
NJ8.png
NR.png
NR1.png
NR2.png
NR3.png
NR4.png
NR5.png
NR7.png
PD1.png
PD2.png
PD3.png
PD4.png
PD5.png
PD7.png
PD8.png
TC2.png
TC3.png
TC5.png
TC6.png
TC7.png
TC8.png
TC9.png
TS1.png
TS2.png
TS3.png
TS4.png
TS5.png
TS6.png
TS7.pn

<h3>CCR<h3>

In [9]:
#To switch from the file name (for example: “FANA_0.jpg”) to its identity (for example: “FANA”)
ccr_label_fn = lambda img: img.split("_")[0]

ccr_model, ccr_class_names = load_model(ccr_models_fc_path, ccr_train_dataset, device, False)
ccr_accuracy = evaluate_model(ccr_class_names, ccr_test_dataset, ccr_model, ccr_label_fn, False)
print(f"CCR Accuracy: {ccr_accuracy:.4f}")

CCR Accuracy: 0.6464
